# China Real Estate Demand Prediction - Lab Report

## Competition Overview
This notebook presents a comprehensive solution to the Kaggle competition "China Real Estate Demand Prediction". The goal is to predict monthly new house transaction amounts across 96 sectors in China.

**Final Leaderboard Score: 0.56248** (Geometric Mean + Seasonality)

### Evaluation Metric
The competition uses a custom two-stage MAPE-based metric:
1. Calculate MAPE for each prediction
2. Compute `good_rate` = % of predictions with APE ≤ 100%
3. If `good_rate` < 0.3: score = 0
4. Otherwise: score = `good_rate` × mean(1/(1 + MAPE) for good predictions)

This metric heavily penalizes predictions when true values are zero or near-zero.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Add parent directory to path for imports
import sys
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

from src.data import DatasetPaths, load_all_training_tables, load_test, prepare_train_target, explode_test_id
from src.features import build_time_lagged_features
from src.models import competition_score

print("Libraries imported successfully!")


## 1. Data Loading and Exploration


In [ ]:
# Load all training data
paths = DatasetPaths(root_dir=str(ROOT))
train_data = load_all_training_tables(paths)

print("Available training tables:")
for name, df in train_data.items():
    print(f"  {name}: {df.shape}")

# Load test data
test_df = load_test(paths)
print(f"\nTest data shape: {test_df.shape}")

# Prepare target variable
target_wide, target_long = prepare_train_target(train_data['new_house_transactions'])
print(f"\nTarget matrix shape: {target_wide.shape} (months × sectors)")


### 1.1 Target Variable Distribution


In [ ]:
# Analyze target distribution
all_values = target_wide.values.flatten()
all_values = all_values[~np.isnan(all_values)]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution of all values
axes[0].hist(all_values, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Transaction Amounts')
axes[0].set_xlabel('Amount')
axes[0].set_ylabel('Frequency')
axes[0].set_yscale('log')

# Distribution of non-zero values
non_zero = all_values[all_values > 0]
axes[1].hist(np.log1p(non_zero), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Log Distribution of Non-Zero Values')
axes[1].set_xlabel('log(Amount + 1)')
axes[1].set_ylabel('Frequency')

# Zero vs Non-zero
zero_counts = [(all_values == 0).sum(), (all_values > 0).sum()]
axes[2].pie(zero_counts, labels=['Zero', 'Non-Zero'], autopct='%1.1f%%', colors=['red', 'green'])
axes[2].set_title('Zero vs Non-Zero Values')

plt.tight_layout()
plt.show()

print(f"Statistics:")
print(f"  Total values: {len(all_values)}")
print(f"  Zero values: {zero_counts[0]} ({100*zero_counts[0]/len(all_values):.1f}%)")
print(f"  Mean (non-zero): {non_zero.mean():.2f}")
print(f"  Median (non-zero): {np.median(non_zero):.2f}")


## 2. Model Benchmarking Results

### 2.1 Baseline Models (Ridge Regression)


In [ ]:
# Load benchmark results
ridge_results = pd.read_csv(ROOT / 'reports' / 'cv_ridge_results.csv')
advanced_results = pd.read_csv(ROOT / 'reports' / 'cv_advanced_models.csv')

# Display Ridge results
print("Ridge Regression Results (Time Series CV):")
print(ridge_results.to_string(index=False))

# Plot Ridge hyperparameter curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(ridge_results['alpha'], ridge_results['score_mean'], 'o-', label='Score')
axes[0].fill_between(ridge_results['alpha'], 
                     ridge_results['score_mean'] - ridge_results['score_std'],
                     ridge_results['score_mean'] + ridge_results['score_std'],
                     alpha=0.3)
axes[0].set_xlabel('Alpha (Regularization)')
axes[0].set_ylabel('Competition Score')
axes[0].set_xscale('log')
axes[0].set_title('Ridge: Score vs Alpha')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ridge_results['alpha'], ridge_results['mape_mean'], 'o-', color='red', label='MAPE')
axes[1].set_xlabel('Alpha (Regularization)')
axes[1].set_ylabel('MAPE')
axes[1].set_xscale('log')
axes[1].set_title('Ridge: MAPE vs Alpha')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 2.2 Advanced Models (Gradient Boosting)


In [ ]:
# Display advanced model results
print("Advanced Models Results (Time Series CV):")
print(advanced_results.to_string(index=False))

# Compare models
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Score comparison
models = advanced_results['model'].values
scores = advanced_results['score_mean'].values
axes[0].bar(models, scores, color=['blue', 'green', 'orange'])
axes[0].set_ylabel('Competition Score')
axes[0].set_title('Model Comparison: Competition Score')
axes[0].set_ylim(0, max(scores) * 1.1)
for i, v in enumerate(scores):
    axes[0].text(i, v + 0.01, f'{v:.3f}', ha='center')

# RMSE comparison
rmse = advanced_results['rmse_mean'].values
axes[1].bar(models, rmse, color=['blue', 'green', 'orange'])
axes[1].set_ylabel('RMSE')
axes[1].set_title('Model Comparison: RMSE')
for i, v in enumerate(rmse):
    axes[1].text(i, v + 500, f'{v:.0f}', ha='center')

plt.tight_layout()
plt.show()

# Best model
best_idx = advanced_results['score_mean'].idxmax()
print(f"\nBest Model: {advanced_results.loc[best_idx, 'model']}")
print(f"  Score: {advanced_results.loc[best_idx, 'score_mean']:.4f} ± {advanced_results.loc[best_idx, 'score_std']:.4f}")
print(f"  RMSE: {advanced_results.loc[best_idx, 'rmse_mean']:.0f}")
print(f"  MAPE: {advanced_results.loc[best_idx, 'mape_mean']:.2f}")


## 3. Bayesian Optimization with Optuna

We performed Bayesian hyperparameter optimization using Optuna for XGBoost:


In [ ]:
# Load Optuna results
import json
with open(ROOT / 'reports' / 'xgb_optuna_best.json', 'r') as f:
    optuna_results = json.load(f)

print("Optuna Bayesian Optimization Results:")
print(f"Best Score: {optuna_results['best_score']:.4f}")
print("\nOptimized Hyperparameters:")
for param, value in optuna_results['best_params'].items():
    if isinstance(value, float):
        print(f"  {param}: {value:.4f}")
    else:
        print(f"  {param}: {value}")

# Compare with baseline
baseline_xgb_score = advanced_results[advanced_results['model'] == 'XGBoost']['score_mean'].values[0]
improvement = (optuna_results['best_score'] - baseline_xgb_score) / baseline_xgb_score * 100

print(f"\nImprovement over baseline XGBoost: {improvement:.1f}%")


## 4. Key Insights and Challenges

### 4.1 Data Challenges
- **15.5% zero values** in the target variable
- **Missing sector 95** in training data but present in test data
- **High variance** in transaction amounts (50 to 606,407)

### 4.2 Metric Challenges
The competition metric severely penalizes predictions when true values are zero:
- If APE > 100% for more than 30% of predictions → Score = 0
- Predicting any positive value when true is 0 → Infinite APE

### 4.3 Solutions Implemented
1. **Time-lagged features**: lag_1, lag_2, lag_3, lag_6, lag_12
2. **Rolling statistics**: 3, 6, 12-month rolling means and geometric means
3. **Zero detection**: Random Forest classifier to identify zero transactions
4. **Ensemble methods**: XGBoost, LightGBM, CatBoost
5. **Bayesian optimization**: Optuna for hyperparameter tuning
6. **Conservative predictions**: Heavy dampening to avoid large errors

## 5. Conclusion

This competition demonstrates the challenges of real-world time series prediction with:
- Sparse data with many zeros
- Strict evaluation metrics
- Missing sectors in training data

Our best local CV score was ~0.50 using XGBoost with Optuna tuning. The main difficulty lies in handling zero values correctly - the metric's design makes it extremely challenging to achieve high scores when the data contains many zero transactions.

### Future Improvements
1. More sophisticated zero detection models
2. Sector-specific models for high-variance sectors
3. External data incorporation (economic indicators, seasonality)
4. Advanced time series models (LSTM, Prophet)


## 7. Final Results - Kaggle Leaderboard

After extensive experimentation with various approaches, we achieved our best results using proven baseline methods from the competition examples:

### Submission Results

| Method | Public Score | Description |
|--------|-------------|-------------|
| **Geometric Mean + Seasonality** | **0.56248** | Best score - Geometric mean with December boost |
| Geometric Mean Baseline | 0.55528 | 6-month geometric mean with zero guard |
| Simple Median | 0.21591 | Conservative median-based approach |
| XGBoost (Optuna-tuned) | 0.00000 | Complex model failed due to zero predictions |
| XGBoost (Basic) | 0.00000 | Failed due to metric sensitivity |
| Ridge Regression | 0.00000 | Linear model couldn't handle zeros |

### Key Success Factors

1. **Geometric Mean**: More robust than arithmetic mean for skewed distributions
2. **Zero Guard**: Critical - if any of last 6 months was zero, predict zero
3. **Seasonality**: December typically shows 30% higher transactions
4. **Simplicity**: Complex models failed due to metric's extreme sensitivity

### Why Complex Models Failed

The competition's two-stage metric severely penalizes any prediction with APE > 100%, especially when true values are zero. Our XGBoost and Ridge models, despite good local CV scores (~0.35-0.50), scored 0.0 on the leaderboard because:
- They occasionally predicted non-zero for zero sectors
- The penalty for these errors overwhelmed any gains from accurate predictions
- The metric requires extremely conservative predictions

### Final Approach Details

```python
# Winning approach: Geometric Mean + Seasonality
1. Calculate 6-month geometric mean for each sector
2. Apply zero guard (if any recent month = 0, predict 0)  
3. Boost December predictions by 1.3x based on historical patterns
4. Result: 0.56248 public score
```

This competition demonstrates that understanding the evaluation metric is crucial - sometimes simpler, more conservative approaches outperform complex machine learning models.


In [ ]:
# Final implementation - Geometric Mean with Seasonality
# This approach achieved 0.56248 on the public leaderboard

def geometric_mean_with_seasonality(train_data, test_data):
    """
    Implements the winning approach: geometric mean with December seasonality boost.
    """
    # Parameters (tuned based on competition examples)
    t1 = 6  # months for geometric mean
    t2 = 6  # months to check for zeros
    december_boost = 1.3  # boost factor for December
    
    # Calculate base predictions
    last_months = train_data.tail(t1)
    geo_mean = np.exp(np.log(last_months.replace(0, np.nan)).mean(axis=0, skipna=True))
    geo_mean = geo_mean.fillna(0)
    
    # Apply zero guard
    zero_mask = train_data.tail(t2).min(axis=0) == 0
    geo_mean[zero_mask] = 0
    
    # Apply December boost for December predictions
    predictions = []
    for month, sector in test_data:
        pred = geo_mean[sector]
        if month == 'December':
            pred *= december_boost
        predictions.append(pred)
    
    return predictions

print("Winning approach implemented successfully!")
